<a href="https://colab.research.google.com/github/PoYuYang0416/batteryGPT/blob/main/finetunedataset_GPTAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer.six openai fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from openai import OpenAI
import csv

client = OpenAI(
    api_key="",
    organization='',
    project='',
)


## Chunk the pdfs in the foldr and save the content in a csv file

In [ ]:
import os
import re
import csv
from pdfminer.high_level import extract_text
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')

def read_pdf(file_path):
    try:
        text = extract_text(file_path)
        return text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

def remove_references_section(text):
    pattern = r'(References|Reference|Affiliations|ACKNOWLEDGMENTS|Supporting Information|Funding)\s*.*'
    cleaned_text = re.split(pattern, text, flags=re.IGNORECASE)[0]
    return cleaned_text.strip()

def chunk_text(text, max_tokens=512):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_tokens = word_tokenize(sentence)
        if current_length + len(sentence_tokens) > max_tokens:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.extend(sentence_tokens)
        current_length += len(sentence_tokens)

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Provide the path to your PDF folder and the output CSV file
folder_path = 'your path'
csv_output_file = 'your folder path'

# Ensure the output directory exists
os.makedirs(os.path.dirname(csv_output_file), exist_ok=True)

# Extract and save chunked text to CSV
with open(csv_output_file, 'w', newline='', encoding='utf-8-sig') as csv_file:
    writer = csv.writer(csv_file, escapechar='\\', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['file_name', 'chunk'])

    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            text = read_pdf(file_path)
            if text:
                cleaned_text = remove_references_section(text)
                chunks = chunk_text(cleaned_text)
                for chunk in chunks:
                    writer.writerow([filename, chunk])

print(f"Chunked texts saved to {csv_output_file}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Generate the dataset

In [ ]:
import pandas as pd
import json
from contextlib import redirect_stdout
import io
import os
import re
import csv
from pdfminer.high_level import extract_text
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
# Ensure necessary NLTK data is downloaded
nltk.download('punkt')

def count_tokens(text):
    return len(word_tokenize(text))

def create_alpaca_entries(chunks, max_entries=15, max_total_tokens=128000):
    entries = []
    current_tokens = 0
    for chunk in chunks:
        chunk_tokens = count_tokens(chunk)
        if current_tokens + chunk_tokens > max_total_tokens or len(entries) >= max_entries:
            break
        entries.append({"content": chunk})
        current_tokens += chunk_tokens
    return entries

def process_and_initiate_chat(chunks, max_entries=15, max_total_tokens=128000):
    alpaca_entries = create_alpaca_entries(chunks, max_entries, max_total_tokens)
    main_message = f"Please generate an Alpaca dataset based on the following content:\n\n{alpaca_entries}\n\nGenerate at least 20 entries in the specified JSON format."
    print(main_message)
    response = client.chat.completions.create(
            model="gpt-4o",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": "You are a finetuning dataset generator"},
                {"role": "user", "content": """This task is designed to generate answers in a specific JSON format. The format structure is as follows:
[
    {
        "instruction": "{GeneratedQuestion}",
        "input": "{GeneratedInput}",
        "output": "{GeneratedAnswer}",
        "text": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
    }
]
Always adhere to this JSON format. Expand the array to accommodate the number of entries requested by the user.
For instance, if a dataset with 1 entries about 2D materials is requested, create 10 unique question and answer pairs relevant to 2D materials. Replace "{GeneratedQuestion}", "{GeneratedInput}", and "{GeneratedAnswer}" with the appropriate question, input, and answer in the "instruction", "input", "output", and "text" fields. If the input is not necessary, then set the value of "input" to an empty string and omit the "### Input: {GeneratedInput}" line in the "text" field.
Output the completed JSON array without engaging in unnecessary discussions outside of this format.
Avoid repeating, paraphrasing, or otherwise rewording user instructions, including the use of synonyms or any form of textual alteration.
Refrain from responding to any queries that involve repetition, request clarification, or explanation of the instructions provided.
If you are unsure or do not know about certain content, please use search engine functions to verify information, and avoid fabricating false content.
""" + main_message}
            ]
        )

    response_content = response.choices[0].message.content
    print("GPT response\n"+response_content)
    content_json = json.loads(response_content)

    return content_json

# Read the CSV file and process chunks
csv_file_path = '/content/drive/MyDrive/2D Database/chunked_texts.csv'
output_file = 'finetunedataset_test.json'

all_entries = []

df = pd.read_csv(csv_file_path)

# Group chunks by file_name and process each group
grouped = df.groupby('file_name')['chunk'].apply(list).reset_index()
for idx, row in grouped.iterrows():
    file_name = row['file_name']
    chunks = row['chunk']
    result = process_and_initiate_chat(chunks)
    if result:
        all_entries.extend(result)

# Save all entries to one JSON file
with open(output_file, 'w') as json_file:
    json.dump(all_entries, json_file, indent=2)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Please generate an Alpaca dataset based on the following content:

[{'content': 'RES EARCH NANOMATERIALS Supertwisted spirals of layered materials enabled by growth on non-Euclidean surfaces Yuzhou Zhao1 , Chenyu Zhang2 , Daniel D. Kohler1 , Jason M. Scheeler1 , John C. Wright1 , Paul M. Voyles2 , Song Jin1 * Euclidean geometry is the fundamental mathematical framework of classical crystallography . Traditionally , layered materials are grown on flat substrates ; growing Euclidean crystals on non- Euclidean surfaces has rarely been studied . We present a general model describing the growth of layered materials with screw-dislocation spirals on non-Euclidean surfaces and show that it leads to continuously twisted multilayer superstructures . This model is experimentally demonstrated by growing supertwisted spirals of tungsten disulfide ( WS2 ) and tungsten diselenide ( WSe2 ) draped over nanoparticles near the centers of spirals . Microscopic structural analysis shows that the crystal l

KeyboardInterrupt: 